In [ ]:
import cv2
import pandas as pd
import numpy as np

def preprocessing(val=True):
    
    data = pd.read_csv('data/x_train_gr_smpl.csv')
    labels = pd.read_csv('data/y_train_smpl.csv')

    if(val):
        ## ---------------- Data preparation ---------------- ##
        X_train = []
        for i in range(data.shape[0]):
            img = np.uint8(data.iloc[i])
            edited = cv2.Canny(img, 10, 30)
            edited = cv2.GaussianBlur(edited, (5, 5), 0)
            X_train.append(edited.reshape((1,-1))[0])

        data = pd.DataFrame(X_train)
        ## -------------------------------------------------- ##

    return data, labels

In [1]:
# Point 5: Deeper analysis of the data

top_2_array = set()
top_5_array = set()
top_10_array = set()

bottom_2_array = set()
bottom_5_array = set()
bottom_10_array = set()

data, _ = preprocessing()

for i in range(10):
    data = data.reindex(np.arange(data.shape[0]))
    labels = pd.read_csv(f'data/y_train_smpl_{i}.csv')
    data['label'] = labels
    data = data.sample(frac=1)
    
    corr_label = data.drop("label", axis=1).apply(lambda x: x.corr(data.label))
    corr_label = [(index, abs(corr_val), i) for index, corr_val in enumerate(corr_label)]
    corr_label = sorted(corr_label, key=lambda tup: tup[1], reverse=True)  # Order by correlation value
            
    for i, tup in enumerate(corr_label[:10]):
        if i < 2:
            top_2_array.add(tup[0])
        if i < 5:
            top_5_array.add(tup[0])
        if i < 10:
            top_10_array.add(tup[0]) #, tup[2]))
            
    for i, tup in enumerate(corr_label[-10:]):
        if i < 2:
            bottom_2_array.add(tup[0])
        if i < 5:
            bottom_5_array.add(tup[0])
        if i < 10:
            bottom_10_array.add(tup[0])

In [10]:
# Point 6: Try to improve the classification

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, classification_report

data, labels = preprocessing()

In [11]:
data_top_2 = data[data.columns[list(top_2_array)]].copy(deep=True)
data_top_2['label'] = labels

X = data_top_2.iloc[:, :-1]  # All columns but the label
y = data_top_2['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

gnb = GaussianNB()

y_pred = gnb.fit(X_train, y_train).predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.63      0.42      0.51       478
           1       0.54      0.34      0.42       591
           2       0.34      0.62      0.44       132
           3       0.66      0.88      0.75       417
           4       0.87      0.74      0.80       717
           5       0.83      0.75      0.79       708
           6       0.50      0.83      0.62       245
           7       0.23      0.90      0.37        86
           8       0.87      0.77      0.82       704
           9       0.57      0.42      0.48       100

    accuracy                           0.67      4178
   macro avg       0.60      0.67      0.60      4178
weighted avg       0.71      0.67      0.67      4178



In [12]:
data_top_5 = data[data.columns[list(top_5_array)]].copy(deep=True)
data_top_5['label'] = labels

X = data_top_5.iloc[:, :-1]
y = data_top_5['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

gnb = GaussianNB()

y_pred = gnb.fit(X_train, y_train).predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.56      0.60      0.58       448
           1       0.61      0.36      0.45       624
           2       0.36      0.69      0.48       131
           3       0.69      0.92      0.79       424
           4       0.88      0.76      0.82       697
           5       0.89      0.81      0.85       701
           6       0.70      0.84      0.76       254
           7       0.30      0.86      0.45        72
           8       0.91      0.84      0.87       717
           9       0.69      0.71      0.70       110

    accuracy                           0.72      4178
   macro avg       0.66      0.74      0.67      4178
weighted avg       0.75      0.72      0.73      4178



In [13]:
data_top_10 = data[data.columns[list(top_10_array)]].copy(deep=True)
data_top_10['label'] = labels

X = data_top_10.iloc[:, :-1]
y = data_top_10['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

gnb = GaussianNB()

y_pred = gnb.fit(X_train, y_train).predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.61      0.62      0.61       493
           1       0.59      0.40      0.48       588
           2       0.47      0.70      0.56       143
           3       0.76      0.95      0.84       421
           4       0.90      0.86      0.88       681
           5       0.95      0.86      0.90       721
           6       0.77      0.87      0.82       266
           7       0.52      0.90      0.66        77
           8       0.91      0.88      0.89       679
           9       0.67      0.85      0.75       109

    accuracy                           0.77      4178
   macro avg       0.71      0.79      0.74      4178
weighted avg       0.78      0.77      0.77      4178



In [14]:
confusion_matrix(y_test, y_pred)

array([[306,  96,  23,  19,   5,   1,  18,  13,   5,   7],
       [171, 238,  69,  20,  15,   5,  49,   7,   8,   6],
       [  9,  14, 100,   1,   1,   0,   1,   5,   9,   3],
       [  0,  11,   5, 399,   2,   0,   0,   0,   2,   2],
       [  0,   4,   7,  42, 588,  24,   0,   6,  10,   0],
       [  1,   9,   2,  15,  35, 617,   0,  10,  20,  12],
       [  8,  20,   1,   5,   0,   0, 232,   0,   0,   0],
       [  0,   0,   1,   0,   2,   0,   0,  69,   1,   4],
       [  6,  11,   4,  27,   5,   2,   0,  18, 595,  11],
       [  3,   0,   2,   0,   2,   0,   0,   4,   5,  93]])

# 7. Conclusions (Draft)

### Which streets signs are harder to recognise?
Based on the recall measure labels 1 and 8 are the harsdest to recognize.
### Which street signs are most easily confused?
Based on the confusion matrix the most confused sign is the label 2 and, in order, it's mistaken for labels 1, 8, 0 and 4. 
### Which attributes (fields) are more reliable and which are less reliable in classification of street signs?
By looking at top_2_array we can identify the most reliable features (pixels). By doing some operations like the module we can see those pixels are closer to the centre of the image (or at least don't belong to the borders).
Regarding the less reliable pixels we consider the ones with the lowest correlation value with respect to the label.
### What was the purpose of Tasks 5 and 6?
Prepare the data in a way that we keep the most relevant attributes.
### What would happen if the data sets you used in Tasks 4, 5 and 6 were not randomised?
It doens't happen anything, the results are the same.
### What would happen if there is cross-correlation between the non-class attributes?
There is actually cross-correlation between pixels that are close to each other. As a consecuence of that, if two pixels are correlated (adjacent) to each other, and one of them is included in the top 10 of the most correlated pixels with the label, the other is likely to be in the top 10 as well.